In [2]:
import pandas as pd

In [13]:
# Load the CSV file
df = pd.read_csv(r"C:\Users\David\Documents\PHASE 4 PROJECT\PHASE 5 FINAL PROJECT\submission_format.csv")

# Display the first few rows
print(df.head())

    uid  year  composite_score
0  abxu  2016                0
1  aeol  2016                0
2  aeol  2021                0
3  afnb  2016                0
4  afnb  2021                0


### 1. Exploratory Data Analysis (EDA)

#### a) View Summary Statistics

In [14]:
print(df.describe())


              year  composite_score
count  1105.000000           1105.0
mean   2019.162896              0.0
std       2.411604              0.0
min    2016.000000              0.0
25%    2016.000000              0.0
50%    2021.000000              0.0
75%    2021.000000              0.0
max    2021.000000              0.0


#### b) Check for Missing Values


In [15]:
print(df.isnull().sum())


uid                0
year               0
composite_score    0
dtype: int64


There are no missing values in the dataset

### 2. Data Visualization